In [690]:
"""
скрипт для генерации рабочих программ ПО
"""

'\nскрипт для генерации рабочих программ ПО\n'

In [691]:
import tkinter
import numpy as np
import sys
import pandas as pd
import os
from docxtpl import DocxTemplate
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import openpyxl
import time
import datetime
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [692]:
def convert_to_int(cell):
    """
    Метод для проверки значения ячейки
    :param cell: значение ячейки
    :return: число в формате int
    """
    if cell is np.nan:
        return 0
    if cell.isdigit():
        return int(cell)
    else:
        return 0

In [693]:
def convert_date(cell):
    """
    Функция для конвертации даты в формате 1957-05-10 в формат 10.05.1957(строковый)
    """

    try:
        string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
        return string_date
    except:
        return 'Проверьте правильность написания даты. Пример правильного написания - 02.04.2023'


In [694]:
name_file_template_educ_program_po = 'data/Автошаблон_ПО_ЦОПП_25072023.docx'
path_to_end_folder_obraz_program_po = 'data'
name_file_data_obraz_program_po = 'data/Для_автозаполнения_ОП_ПО_25072023.xlsx'

In [695]:
# Открываем файл
wb = openpyxl.load_workbook(name_file_data_obraz_program_po)

In [696]:
name_sheet_up=wb.sheetnames[0] # получаем название листа с учебным планом
name_sheet_data = wb.sheetnames[1] # получаем название листа с данными программы

In [697]:
target_value = 'ИТОГО'

# Поиск значения в выбранном столбце
column_number = 1  # Номер столбца, в котором ищем значение (например, столбец A)
target_row = None  # Номер строки с искомым значением

for row in wb[name_sheet_up].iter_rows(min_row=1, min_col=column_number, max_col=column_number):
    cell_value = row[0].value
    if cell_value == target_value:
        target_row = row[0].row
        break

        


In [698]:
if not target_row:
    print('В первой колонке отсутствует слово ИТОГО!!!')

# если значение найдено то считываем нужное количество строк и  7 колонок
df_up = pd.read_excel(name_file_data_obraz_program_po,sheet_name=name_sheet_up,nrows=target_row,usecols='A:G',dtype=str)
    

In [699]:
df_up.iloc[:,1:6] = df_up.iloc[:,1:6].applymap(convert_to_int) #1) Приводим к инту колонки 2-6 

In [700]:
# Заполняем возможные пустые строки
df_up['Наименование_раздела'] = df_up['Наименование_раздела'].fillna('Не заполнено название раздела')
# Очищаем от возможнных пробелов 
df_up['Наименование_раздела'] = df_up['Наименование_раздела'].apply(lambda x:x.strip())

In [701]:
df_up

,Наименование_раздела,Трудоемкость,Теоретич_час,Практич_час,Практика_час,СРС_час,Форма_контроля
0,Нормативно-правовые основы деятельности по обе...,4,4,0,0,0,х
1,Первая помощь,22,8,14,0,0,х
2,Специальная подготовка,24,24,0,0,0,Зачет
3,Отработка практических приемов и способов поис...,16,0,16,0,0,Экзамен
4,Итоговая аттестация,8,0,2,6,0,Квалификационный экзамен
5,ИТОГО,74,36,32,6,0,х


In [702]:
# Создаем датафрейм учебной программы без учета строки ИТОГО для таблиц краткой аннотации
df_up = df_up[df_up['Наименование_раздела'] != 'ИТОГО']

In [703]:
# получаем единичные значения из листа с данными
single_row_df = pd.read_excel(name_file_data_obraz_program_po,sheet_name=name_sheet_data,nrows=1,usecols='A:K')
single_row_df.iloc[:,8] = single_row_df.iloc[:,8].apply(convert_date) # обрабатываем колонку с датой

In [704]:
single_row_df

,Тип_программы,Код_профессии,Наименование_профессии,Форма_обучения,Срок_освоения,Год,Профессиональный_стандарт,Рег_номер_ПС,Дата_приказа_МИНТРУДА,Номер_приказа_МИНТРУДА,Разряд
0,профессии рабочего,13495,Матрос-спасатель,очная,72,2023,Спасатель на акватории,1410,07.12.2020,862н,2


In [705]:
# Очищаем от лишнего поля которые заполняет пользователь
# Заполняем возможные пустые строки
single_row_df['Наименование_профессии'] = single_row_df['Наименование_профессии'].fillna('Не заполнено !!!')
# Очищаем от возможнных пробелов 
single_row_df['Наименование_профессии'] = single_row_df['Наименование_профессии'].apply(lambda x:x.strip())

single_row_df['Профессиональный_стандарт'] = single_row_df['Профессиональный_стандарт'].fillna('Не заполнено !!!')
# Очищаем от возможнных пробелов 
single_row_df['Профессиональный_стандарт'] = single_row_df['Профессиональный_стандарт'].apply(lambda x:x.strip())





In [706]:
single_row_df

,Тип_программы,Код_профессии,Наименование_профессии,Форма_обучения,Срок_освоения,Год,Профессиональный_стандарт,Рег_номер_ПС,Дата_приказа_МИНТРУДА,Номер_приказа_МИНТРУДА,Разряд
0,профессии рабочего,13495,Матрос-спасатель,очная,72,2023,Спасатель на акватории,1410,07.12.2020,862н,2


In [707]:
# получаем датафрейм с технологиями обучения
tech_df = pd.read_excel(name_file_data_obraz_program_po,sheet_name=name_sheet_data,usecols='L:O')

In [708]:
tech_df.dropna(thresh=2,inplace=True) # очищаем от строк в которых не заполнены 2 колонки

In [709]:
tech_df['Разработчики_программы'] = tech_df['Разработчики_программы'].fillna('Не заполнено !!!')
# Очищаем от возможнных пробелов 
tech_df['Характеристика_технологии_обучения'] = tech_df['Характеристика_технологии_обучения'].apply(lambda x:x.strip())
tech_df['Технологии_обучения'] = tech_df['Технологии_обучения'].apply(lambda x:x.strip())
tech_df['Разработчики_программы'] = tech_df['Разработчики_программы'].apply(lambda x:x.strip())


In [710]:
tech_df

,Уровни_квалификации,Технологии_обучения,Характеристика_технологии_обучения,Разработчики_программы
0,2.0,Проблемно-поисковые технологии,"технологии, основанные на создании в учебном п...","Протасова Е.Н., методолог ЦОПП РБ"
1,3.0,"Практико-ориентированные, деятельностные техно...",технологии направлены на формирование системы ...,"Иванов И.И., преподаватель ГАПОУ ""РМК"""
2,NaN,Личностно-ориентированные технологии для слуша...,обеспечивают чувство комфортности педагога и с...,Истомина


In [711]:
tech_df['Уровни_квалификации'] = tech_df['Уровни_квалификации'].fillna(0)
tech_df['Уровни_квалификации'] = tech_df['Уровни_квалификации'].astype(int)

In [712]:
# создаем переменную для уровней квалификации
levels_qual = tech_df['Уровни_квалификации'].to_list()
levels_qual = list(filter(lambda x:x !=0,levels_qual))
levels_qual = list(map(str,levels_qual))


In [713]:
tech_df

,Уровни_квалификации,Технологии_обучения,Характеристика_технологии_обучения,Разработчики_программы
0,2,Проблемно-поисковые технологии,"технологии, основанные на создании в учебном п...","Протасова Е.Н., методолог ЦОПП РБ"
1,3,"Практико-ориентированные, деятельностные техно...",технологии направлены на формирование системы ...,"Иванов И.И., преподаватель ГАПОУ ""РМК"""
2,0,Личностно-ориентированные технологии для слуша...,обеспечивают чувство комфортности педагога и с...,Истомина


In [714]:
# Конвертируем датафрейм с описанием программы в список словарей
data_program = single_row_df.to_dict('records')

In [715]:
data_program

[{'Тип_программы': 'профессии рабочего',
  'Код_профессии': 13495,
  'Наименование_профессии': 'Матрос-спасатель',
  'Форма_обучения': 'очная',
  'Срок_освоения': 72,
  'Год': 2023,
  'Профессиональный_стандарт': 'Спасатель на акватории',
  'Рег_номер_ПС': 1410,
  'Дата_приказа_МИНТРУДА': '07.12.2020',
  'Номер_приказа_МИНТРУДА': '862н',
  'Разряд': 2}]

In [716]:
context = data_program[0] 

In [717]:
context['Уровни_квалификации'] = ','.join(levels_qual)


In [718]:
# Добавляем датафреймы
context['lst_tech'] = tech_df.to_dict('records') # образовательные технологии
context['up_lst'] = df_up.to_dict('records') # учебный план
context['lst_dev'] = tech_df['Разработчики_программы'].tolist()

In [719]:
doc = DocxTemplate(name_file_template_educ_program_po)
# Создаем документ
doc.render(context)
# сохраняем документ
# название программы
name_prof = single_row_df['Наименование_профессии'].tolist()[0]
razr = single_row_df['Разряд'].tolist()[0]
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
doc.save(
    f'{path_to_end_folder_obraz_program_po}/Программа ПО {name_prof} {razr} разряда {current_time}.docx')

In [720]:
print('Lindy Booth')

Lindy Booth
